In [1]:
import requests
import time
import os, json
import csv
import concurrent.futures
# from functools import partial
import pandas as pd

In [27]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
request_payload_file = '/data/Magicoder_payloads.txt'
report_path = '/data/vLLM_xyzConcurrency_1gpu.csv'
df = pd.DataFrame(columns=['Prompt_No', 'Prompt', 'Response', 'Response_status', 'Time (seconds)'])

In [16]:
payload = {"prompt": "Problem: Create an animated dropdown menu in CSS that appears when a button is clicked.Input: Button element selector, dropdown menu element selector.Output: None (animate the dropdown menu).", 
           "max_tokens": 200,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [17]:
def read_payloads(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            prompt = json.loads(line.strip())
            yield prompt['payload']

In [18]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        global df
        start_time = time.time()
        result, a,b= func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Prompt_No':b,'Prompt':a,'Response':result.text, 'Response_status':result.status_code, 'Time (seconds)':f'{execution_time:.6f}'}
        df = df.append(data, ignore_index=True)
        print(f"Execution time: {execution_time:.6f} seconds \n\n")
        return result
    return calculate_execution_time

In [19]:
def save_report(file_path, df):
    print(df)
    df.to_csv(file_path, index=False)

In [14]:
@my_decorator
def inference(thread, payload_data=None):
    headers = {
        'Content-Type': 'application/json'
    }
    
    response = requests.request("POST", url, headers=headers, json=payload_data)
    print(f"Prompt {thread}==> {payload_data['prompt']} \n {response.text}")
    return response, payload_data['prompt'], thread
    
        

In [ ]:
def my_worker():
    prompts = read_payloads(request_payload_file)
    try:
        for line, prompt in enumerate(prompts,start=0):
            a,b,c = inference(line, prompt)
    except Exception as e:
        print(f"Error in thread {line}: {e}")

In [28]:
#Model Inference(Request)
if __name__ == "__main__":
    global df
#     for i in range(1):
#         print(f"loop {i}+++++++++++++++++++++++++++++")
    with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
        futures = [executor.submit(my_worker) for _ in range(30)]
        concurrent.futures.wait(futures)
    save_report(report_path, df)

100
loop 0+++++++++++++++++++++++++++++
Prompt 1==> Problem: Write a function in python to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-4576c90798fe4d2ebb943e61ca8924ad","object":"text_completion","created":522519,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef reverse_string(s):\n    return s[::-1]\n\n# Test the function\nprint(reverse_string(\"Hello, World!\"))  # Output: !dlroW ,olleH","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":31,"total_tokens":82,"completion_tokens":51}}
Execution time: 3.192646 seconds 


Prompt 6==> Problem: Write a function in python to count the number of words in a given string.Input: String s.Output: Integer, the number of words in s. 
 {"id":"cmpl-5d362a9e032948b0ba7b4b8cfabe767b","object":"text_completion","created":522519,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/sna

Prompt 5==> Problem: Write a function in python to generate the nth Fibonacci number.Input: Non-negative integer n.Output: Integer, the nth Fibonacci number. 
 {"id":"cmpl-910c15a7258b443d94f2990ab43545fb","object":"text_completion","created":522519,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\nFibonacci sequence: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, ...\n\nThe Fibonacci sequence is defined as follows:\n\nF(0) = 0, F(1) = 1\n\nFor n > 1, F(n) = F(n-1) + F(n-2)\n\nHere is a Python function that generates the nth Fibonacci number:\n\n```python\ndef fibonacci(n):\n    if n <= 0:\n        return \"Input should be a positive integer.\"\n    elif n == 1:\n        return 0\n    elif n == 2:\n        return 1\n    else:\n        a, b = 0, 1\n       ","logprobs":null,"finish_reason":"length"}],"usage":{"prompt_tokens":43,"total_tokens":243,"completion_tokens":200}}
Execution time: 11.41231

Prompt 18==> Problem: Write a function in java to perform binary search on a sorted list.Input: Sorted list arr and target value target.Output: Integer, the index of target in arr (or -1 if not found). 
 {"id":"cmpl-e9ae030e20574a4294b4d859efbd143e","object":"text_completion","created":522529,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\n```java\npublic class BinarySearch {\n    public static int binarySearch(int[] arr, int target) {\n        int left = 0;\n        int right = arr.length - 1;\n\n        while (left <= right) {\n            int mid = left + (right - left) / 2;\n\n            if (arr[mid] == target) {\n                return mid;\n            } else if (arr[mid] < target) {\n                left = mid + 1;\n            } else {\n                right = mid - 1;\n            }\n        }\n\n        return -1;\n    }\n\n    public static void main(String[] args) {\n        i

Prompt 23==> Problem: Write a function in c++ to check if a given string is a palindrome.Input: String s.Output: Boolean, indicating whether s is a palindrome. 
 {"id":"cmpl-56ddd7f25a9c4657af967f414b3db515","object":"text_completion","created":522536,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nSolution:\n\n```cpp\n#include <iostream>\n#include <string>\n\nbool isPalindrome(std::string s) {\n    int start = 0;\n    int end = s.length() - 1;\n\n    while (start < end) {\n        if (s[start] != s[end]) {\n            return false;\n        }\n        start++;\n        end--;\n    }\n    return true;\n}\n\nint main() {\n    std::string s;\n    std::cout << \"Enter a string: \";\n    std::cin >> s;\n\n    if (isPalindrome(s)) {\n        std::cout << \"The string is a palindrome.\" << std::endl;\n    } else {\n        std::cout << \"The string is not a palind","logprobs":null,"finish_reaso

Prompt 41==> Problem: Write a function in Ruby to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-d433b34a938941a8b088fd52de6b8626","object":"text_completion","created":522552,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef reverse_string(s)\n  s.reverse\nend\n\nputs reverse_string(\"Hello World\")","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":32,"total_tokens":62,"completion_tokens":30}}
Execution time: 1.727016 seconds 


Prompt 37==> Problem: Write a function in javascript to count the number of words in a given string.Input: String s.Output: Integer, the number of words in s. 
 {"id":"cmpl-3478de01acfd46af9a19d234694ecb00","object":"text_completion","created":522548,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nfunc

...

Prompt 66==> Problem: Build a function in CSS to populate an HTML table with data from an array of objects.Input: Array of objects, each representing a row in the table.Output: None (update the HTML table). 
 {"id":"cmpl-f2dd49a4a5434957b4074791ce5a898c","object":"text_completion","created":522574,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple example of how you can do this in CSS:\n\n```css\n@keyframes populate-table {\n  0% {\n    opacity: 0;\n  }\n  100% {\n    opacity: 1;\n  }\n}\n\n.populate-table {\n  animation: populate-table 1s;\n}\n```\n\nIn this example, the `populate-table` class is added to the table when the data is ready to be displayed. The animation will gradually increase the opacity of the table from 0 to 1, making it visible.\n\nYou can use JavaScript or jQuery to add the class to the table when the data is ready:\n\n```javascript\nvar data = [\n  {name

Prompt 73==> Problem: Create an animated dropdown menu in react that appears when a button is clicked.Input: Button element selector, dropdown menu element selector.Output: None (animate the dropdown menu). 
 {"id":"cmpl-4c00f5d3679840f2b59bc3202943354e","object":"text_completion","created":522583,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple example of how you can create an animated dropdown menu in React using CSS and JavaScript.\n\n```jsx\nimport React, { useState } from 'react';\n\nconst DropdownMenu = () => {\n  const [isOpen, setIsOpen] = useState(false);\n\n  const toggleDropdown = () => {\n    setIsOpen(!isOpen);\n  };\n\n  return (\n    <div className=\"dropdown\">\n      <button onClick={toggleDropdown}>\n        {isOpen ? 'Close' : 'Open'} Dropdown\n      </button>\n      {isOpen && (\n        <div className=\"dropdown-menu\">\n          <ul>\n            <li>

Prompt 80==> Problem: Implement an autocomplete feature in angular for a search input field.Input: String, user input in the search field.Output: None (update the autocomplete suggestions). 
 {"id":"cmpl-d098687c1359426b8e4ea1c0308fbc4d","object":"text_completion","created":522592,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nSolution:\n\nTo implement an autocomplete feature in Angular, we can use the Angular Material library which provides a directive for autocomplete.\n\nHere is a simple example of how you can implement it:\n\n1. Install Angular Material:\n\n```bash\nnpm install @angular/material @angular/cdk\n```\n\n2. Import the Angular Material modules in your app.module.ts:\n\n```typescript\nimport { MatAutocompleteModule } from '@angular/material/autocomplete';\nimport { MatInputModule } from '@angular/material/input';\nimport { BrowserAnimationsModule } from '@angular/platform-br

Prompt 87==> Problem: Write a function in angular that dynamically creates an HTML element and appends it to the document.Input: String, element type (e.g., 'div', 'button').Output: None (update the HTML document). 
 {"id":"cmpl-021a10d8a7fc4fcf8446be74cdd5a338","object":"text_completion","created":522598,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple function in Angular that creates an HTML element and appends it to the document:\n\n```typescript\nimport { Component, ElementRef, Renderer2 } from '@angular/core';\n\n@Component({\n  selector: 'app-root',\n  templateUrl: './app.component.html',\n  styleUrls: ['./app.component.css']\n})\nexport class AppComponent {\n  constructor(private el: ElementRef, private renderer: Renderer2) {}\n\n  createElement(elementType: string, text: string) {\n    const newElement = this.renderer.createElement(elementType);\n    const textNode 

Prompt 93==> Problem: Create an animated dropdown menu in jQuery that appears when a button is clicked.Input: Button element selector, dropdown menu element selector.Output: None (animate the dropdown menu). 
 {"id":"cmpl-ecc1212a9cc94d3a8f158de753f64fc4","object":"text_completion","created":522606,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple example of how you can create an animated dropdown menu using jQuery:\n\nHTML:\n```html\n<button id=\"dropdown-button\">Click me</button>\n<div id=\"dropdown-menu\" style=\"display: none;\">\n  <ul>\n    <li>Option 1</li>\n    <li>Option 2</li>\n    <li>Option 3</li>\n  </ul>\n</div>\n```\n\nCSS:\n```css\n#dropdown-menu {\n  position: absolute;\n  background-color: #f9f9f9;\n  min-width: 160px;\n  box-shadow: 0px 8px 16px 0px rgba(0,0,0,0.2);\n  z-index: 1;\n}","logprobs":null,"finish_reason":"length"}],"usage":{"prompt_tokens":43,

In [15]:
# #Model Inference(Request) Same Payload
# if __name__ == "__main__":
#     # Using the generator function to read lines 
#     prompts = list(read_payloads(request_payload_file))
#     index=0
#     global df
#     for i in range(1):
#         print(f"loop {i}+++++++++++++++++++++++++++++")
#         with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
#             futures = [executor.submit(inference, srno, payload) for srno in range(100)]
#             concurrent.futures.wait(futures)
#     save_report(report_path, df)

loop 0+++++++++++++++++++++++++++++
Prompt 1==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-f3c65729fed346f39fc4674a9e705cac","object":"text_completion","created":520352,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorted(s1) == sorted(s2)\n\n# Test the function\nprint(are_anagrams(\"listen\", \"silent\"))  # True\nprint(are_anagrams(\"hello\", \"world\"))  # False","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":45,"total_tokens":148,"completion_tokens":103}}
Execution time: 8.118660 seconds 


Prompt 4==> Problem: Write a function in python to check if two 

Prompt 25==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-342c920d2e6f4784ba643e441a2eb74a","object":"text_completion","created":520353,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorted(s1) == sorted(s2)\n\n# Test the function\nprint(are_anagrams(\"listen\", \"silent\"))  # True\nprint(are_anagrams(\"hello\", \"world\"))  # False","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":45,"total_tokens":148,"completion_tokens":103}}
Prompt 29==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicat

Execution time: 7.728093 seconds 


Execution time: 7.605895 seconds 


Execution time: 8.052507 seconds 


Execution time: 7.545251 seconds 

Execution time: 7.825602 seconds 



Execution time: 8.250600 seconds 


Execution time: 7.640745 seconds 


Execution time: 8.151742 seconds 


Execution time: 7.583045 seconds 

Execution time: 7.704802 seconds 



Prompt 30==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-4f203ab7fd1a4bb0ac2a8424fc7c3486","object":"text_completion","created":520360,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorted(s1) == sorted(s2)\n\n# Test the func


Execution time: 7.941365 seconds 

Execution time: 8.571286 seconds 

Execution time: 8.291329 seconds 



Execution time: 8.533391 seconds 


Execution time: 8.407189 seconds 




Execution time: 8.274467 seconds 


Execution time: 8.718619 seconds 


Execution time: 8.113958 seconds 


Execution time: 8.330332 seconds 


Execution time: 8.357188 seconds 

Execution time: 8.330998 seconds 



Prompt 60==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-35bde0b5907648419f139e4988d9a300","object":"text_completion","created":520367,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorte

Execution time: 8.450169 seconds 


Execution time: 8.299783 seconds 


Execution time: 8.828971 seconds 


Prompt 90==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-63c8d8e23f164b1299539102fab91171","object":"text_completion","created":520375,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorted(s1) == sorted(s2)\n\n# Test the function\nprint(are_anagrams(\"listen\", \"silent\"))  # True\nprint(are_anagrams(\"hello\", \"world\"))  # False","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":45,"total_tokens":148,"completion_tokens":103}}
Execution time: 6.317636 se

In [ ]:
def process_data(data):
    print(f"Processing data: {data}")

def worker():
    # Create a new generator instance for each thread
    gen = my_generator()
    
    try:
        for value in gen:
            process_data(value)
    except Exception as e:
        print(f"Error in thread: {e}")
    finally:
        # Ensure proper cleanup (optional)
        print("Thread cleanup")

# Set the number of threads
num_threads = 3

# Outer loop for reiterating
for _ in range(2):  # Adjust the number of iterations as needed
    # Create a ThreadPoolExecutor with the specified number of threads
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        # Submit tasks to the ThreadPoolExecutor
        futures = [executor.submit(worker) for _ in range(num_threads)]

        # Wait for all tasks to complete
        concurrent.futures.wait(futures)